In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/programmers/

/content/drive/My Drive/programmers


In [ ]:
# !unzip *

In [ ]:
!pip install -U efficientnet

In [16]:
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import efficientnet.keras as efn

from keras.models import load_model

In [17]:
train_path = '/content/drive/My Drive/programmers/train/'

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    validation_split=0.2
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical',
    seed=123,
    batch_size=BATCH_SIZE,
    subset='training'
)

validation_generator = validation_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical',
    seed=123,
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 1361 images belonging to 7 classes.
Found 337 images belonging to 7 classes.


In [ ]:
# for x, y in train_generator:
#     print(x.shape, y.shape)
    
#     fig, axes = plt.subplots(2, 5)
#     fig.set_size_inches(15, 10)
#     for i in range(10):
#         axes[i//5, i%5].imshow(x[i])
#         axes[i//5, i%5].axis('off')
#     plt.show()
#     break

In [ ]:
efficientNet = efn.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

x = efficientNet.layers[-1].output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7, activation='softmax')(x)

model = tf.keras.Model(efficientNet.input, x)

for layer in model.layers[:-10]:
  layer.trainable = False
for layer in model.layers[-10:]:
  layer.trainable = True


model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

model.summary()

In [20]:
# os.makedirs('models', exist_ok=True)
start_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

cp_earlystop = EarlyStopping(monitor='val_loss',
                             mode='min',
                             patience=10,
                             verbose=1)

cp_callback = ModelCheckpoint('models/%s-{epoch:02d}-{val_loss:.2f}.h5' % (start_time),
                              monitor='val_loss',
                              save_best_only=True,
                              mode='min', 
                              verbose=1)

In [ ]:
history = model.fit(train_generator,
                    epochs = 150,
                    validation_data = validation_generator,
                    callbacks=[cp_earlystop, cp_callback])

In [24]:
best_model = load_model('models/model_5.h5')
# best_model.save('model_.h5')

In [25]:
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

validation_generator = validation_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode=None,
    seed=123,
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 337 images belonging to 7 classes.


In [27]:
# preds = best_model.predict_generator(validation_generator)
# pred_idx = preds.argmax(axis=-1)

# label = {0 : 'dog', 1 : 'elephant', 2 : 'giraffe', 3 : 'guitar',
#          4 : 'horse', 5 : 'house', 6 : 'person'}
# pred_label = []

# for idx in range(len(pred_idx)):
#   pred_label.append(label[pred_idx[idx]])

In [ ]:
# for x, y in zip(validation_generator, pred_idx):
#     print(x[0].shape, y)
    
#     fig, axes = plt.subplots(4, 5)
#     fig.set_size_inches(15, 10)
#     for i in range(20):
#         axes[i//5, i%5].imshow(x[i])
#         axes[i//5, i%5].axis('off')
#         axes[i//5, i%5].set_title(pred_label[i])
#     plt.show()
#     break

In [29]:
test_path = '/content/drive/My Drive/programmers/test/'

test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    class_mode=None,
    batch_size=BATCH_SIZE
)

Found 350 images belonging to 1 classes.


In [ ]:
preds = best_model.predict_generator(test_generator)
pred_idx = preds.argmax(axis=-1)

In [ ]:
# n_members = 5
# models = []

# for i in range(n_members):
# 	filename = 'models/model_' + str(i + 1) + '.h5'
# 	model = load_model(filename)
# 	models.append(model)

# models

In [ ]:
# sum_preds = [model.predict_generator(test_generator) for model in models]
# sum_preds = np.array(sum_preds)
# summed = np.sum(sum_preds, axis=0)
# pred_idx = summed.argmax(axis=1)

In [50]:
#submission

submission = pd.read_csv('/content/drive/My Drive/programmers/test_answer_sample_.csv')
submission['answer value'] = pred_idx
submission.to_csv('/content/drive/My Drive/programmers/submission_07.csv', index=False)

(350, 2)


,Unnamed: 0,answer value
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
